In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle, sample
import time
import random
import seaborn as sns
import os
import scipy as sp

In [5]:
# ---- POPULATION INITIALISATION ----

def InitialPopulation(cities, population_size):
    '''
    PARAMETERS: 
    - cities: number of vertices (cities) in TSP
    - population_size: the number of randomly generated solutions (routes) we want
    DESCRIPTION: 
    - Initialises population_size randomly generated routes/ solutions to TSP
    RETURNS: 
    - List of lists of random routes taken by induviduals in the population
    '''
    
    # Empty list to store solutions
    population_routes = []

    # Iterates through the population 
    for i in range(population_size):

        # Generates random route through the cities
        route = sample(range(0, cities), cities)

        # Appends random route for that induvidual to the solutions list
        population_routes.append(route)

    return population_routes

In [6]:
# ---- COST FUNCTIONS ----

def Cost(D, C, n):
    '''
    PARAMETERS: 
    - D: Distance Matrix
    - C: Route Vector
    - n: Total number of vertices (cities) in TSP
    DESCRIPTION: 
    - Calculates the cost of a candidate solution
    RETURNS: 
    - Cost of route inputted in the function call
    '''
    
    # Equation as defined in CA Overview
    cost = sum([ D[C[i]][C[i+1]] for i in range(0,n-1)]) + D[C[n-1]][C[0]]

    return cost

def PopulationCosts(D, population, cities):
    '''
    PARAMETERS: 
    - D: Distance Matrix
    - cities: number of vertices (cities) in TSP
    - population: induvidual solutions
    DESCRIPTION: 
    - Calculates fitness/ cost of population of solutions/routes
    RETURNS: 
    - List of lists containing cost of route taken by each solution in population
    '''

    # Empty list to store cost of each route taken by induviduals in population
    population_costs = []

    # Loop calculates cost of route taken by each induvidual in population
    for induvidual in population:
         
         # Calculates cost using cost function
         cost = Cost(D, induvidual, cities)

         # Appends induviduals cost to the population costs
         population_costs.append(cost)

    return population_costs

In [8]:
# ---- SELECTION METHODS ----

def TournamentSelection(D, population, tournament_size, cities):
    '''
    PARAMETERS:
    - D: Distance Matrix
    - population: induvidual solutions
    - tournament_size: the tournament size
    - cities: total number of vertices (cities) in TSP
    DESCRIPTION:
    - Conducts tournament selection twice to pick out two parent solutions
    RETURNS: 
    - Tuple of lists containing routes (genes) of parents selected in the tournaments
    '''

    # First tournament
    tournament1_index = sample(range(len(population)), tournament_size)
    tournament1 = [population[index] for index in tournament1_index]
    # Calculates costs of induviduals in the tournament
    costs1 = PopulationCosts(D, tournament1, cities)
    # Combines the cost with the corresponding routes taken by individuals
    tournament1 = list(zip(tournament1, costs1))
    # Shuffles the tournament information to randomize the order of tied individuals (so that we break ties randomly)
    shuffle(tournament1)
    # Finds the individual with the highest fitness score
    parent1 = max(tournament1, key = lambda x: x[1])

    # Second tournament (analogous to first tournament)
    tournament2_index = sample(range(len(population)), tournament_size)
    tournament2 = [population[index] for index in tournament2_index]
    costs2 = PopulationCosts(D, tournament2, cities)
    tournament2 = list(zip(tournament2, costs2))
    shuffle(tournament2)
    parent2 = max(tournament2, key = lambda x: x[1])

    # Stores the route taken by the fittest induvidual in each tournament 
    parent1 = parent1[0]
    parent2 = parent2[0]
    
    return [parent1, parent2]

In [9]:
# ---- CROSSOVER METHODS ----

def SinglePointCrossover(parents):
    '''
    PARAMETERS:
    - parents: tuple of lists of parent solutions to be used in crossover
    DESCRIPTION:
    - Conducts single point crossover to create two offspring from two parents
    RETURNS: 
    - Tuple of lists of routes (genes) of offspring
    '''

    # Stores genes of parents in variables
    parent1 = parents[0]
    parent2 = parents[1]

    # Here we randomly select a single crossover point
    crossover_point = random.randint(1, len(parent1) - 1)

    # Performs the single point crossover and defines the offspring genomes including constraint that each city must appear uniquely once
    child1 = parent1[:crossover_point] + [city for city in parent2 if city not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [city for city in parent1 if city not in parent2[:crossover_point]]

    # Ensures Crossover adheres to constraint that each city must be visited once exactly in route
    assert len(set(child1)) == len(child1), "City uniqueness violated after crossover"
    assert len(set(child2)) == len(child2), "City uniqueness violated after crossover"

    return [child1, child2]

def OrderedCrossover(parents):
    '''
    PARAMETERS:
    - parents: tuple of lists of parent solutions to be used in crossover
    DESCRIPTION:
    - Conducts ordered crossover to create offspring from two parents
    RETURNS: 
    - Tuple of lists of routes (genes) of offspring
    '''

    # Stores genes of parents in variables
    parent1 = parents[0]
    parent2 = parents[1]

    # Initialises two offspring genomes of the same length as parents with placeholder values -1
    child1 = len(parent1) * [-1]
    child2 = len(parent1) * [-1]

    # Defines random subset of genome to copy from parent 1
    start1 = random.randint(0, len(parent1)-1)
    stop1 = random.randint(start1, len(parent1))

    # Defines random subset of genome to copy from parent 2
    start2 = random.randint(0, len(parent2)-1)
    stop2 = random.randint(start2, len(parent2))

    # Copies the random subset of genes from parent1 and parent2 to child1 and child2 in that same section of genome
    child1[start1:stop1] = parent1[start1:stop1]
    child2[start2:stop2] = parent2[start2:stop2]

    # Fills the remaining genes in child1 with genes from parent2 in the order that they appear while resolving duplicates
    for gene in parent2:

        if gene not in child1:

            child1[stop1 % len(parent2)] = gene
            stop1 += 1

     # Fills the remaining genes in child2 with genes from parent1 in the order that they appear while resolving duplicates
    for gene in parent1:

        if gene not in child2:
            
            child2[stop2 % len(parent1)] = gene
            stop2 += 1

    # Ensures Crossover adheres to constraint that each city must be visited once exactly in route
    assert len(set(child1)) == len(child1), "City uniqueness violated after crossover"
    assert len(set(child2)) == len(child2), "City uniqueness violated after crossover"

    return [child1, child2]

def PartiallyMappedCrossover(parents):
    '''
    PARAMETERS:
    - parents: tuple of lists of parent solutions to be used in crossover
    DESCRIPTION:
    - Conducts partially mapped crossover to create offspring from two parents
    RETURNS: 
    - Tuple of lists of routes (genes) of offspring
    '''

    # Stores genes of parents in variables
    parent1 = parents[0]
    parent2 = parents[1]

    # Initialises two offspring genomes of the same length as parents with placeholder values -1
    child1 = len(parent1) * [-1]
    child2 = len(parent1) * [-1]

    # Defines random subset of genome to use in crossover
    start = random.randint(0, len(parent1)-1)
    stop = random.randint(start, len(parent1))

    # Copies the random subset of genes from parent1 and parent 2 to child1 and child2
    child1[start:stop + 1] = parent1[start:stop + 1]
    child2[start:stop + 1] = parent2[start:stop + 1]

    # We initialise sets to keep track of included cities
    child1_subset_genes = set(child1[start:stop + 1])
    child2_subset_genes = set(child2[start:stop + 1])

    # We fill in the remaining positions with values from parents
    for gene in range(len(parent1)):

        if child1[gene] == -1:

            temp = parent2[gene]

            while temp in child1_subset_genes:

                index = parent2.index(temp)
                temp = parent1[index]

            child1[gene] = temp
            child1_subset_genes.add(temp)

        if child2[gene] == -1:

            temp = parent1[gene]

            while temp in child2_subset_genes:

                index = parent1.index(temp)
                temp = parent2[index]

            child2[gene] = temp
            child2_subset_genes.add(temp)

    # Ensures Crossover adheres to constraint that each city must be visited once exactly in route
    assert len(set(child1)) == len(child1), "City uniqueness violated after crossover"
    assert len(set(child2)) == len(child2), "City uniqueness violated after crossover"

    return [child1, child2]

In [12]:
# ---- MUTATION METHODS ----

def SingleSwapMutation(offspring): 
    '''
    PARAMETERS: 
    - offspring: the tuple of offspring solutions after crossover to be mutated
    DESCRIPTION: 
    - Conducts single swap mutation on the two offspring i.e. selects two random genes (vertex) and swaps their position in genenome
    RETURNS: 
    - Tuple of routes (genes) of two offspring after mutation
    '''

    # Stores genes of parents in variables
    child1 = offspring[0]
    child2 = offspring[1]

    # Randomly select two positions in genome
    position1 = random.choice(range(len(child1)))
    position2 = random.choice(range(len(child2)))

    # Swap genes at the two random positions
    child1[position1], child1[position2] = child1[position2], child1[position1]
    child2[position1], child2[position2] = child2[position2], child2[position1]

    # Ensures mutation adheres to constraint that each city (gene) must be expressed once
    assert len(set(child1)) == len(child1), "City uniqueness violated after mutation"
    assert len(set(child2)) == len(child2), "City uniqueness violated after mutation"

    return [child1, child2]

def InsertionMutation(offspring):
    '''
    PARAMETERS: 
    - offspring: the tuple of offspring solutions after crossover to be mutated
    DESCRIPTION: 
    - Conducts insertion mutation on the two offspring 
    RETURNS: 
    - Tuple of routes (genes) of two offspring after mutation
    '''

    # Stores genes of parents in variables
    child1 = offspring[0]
    child2 = offspring[1]

    # Selects a random element in each child genome
    gene1 = random.choice(child1)
    gene2 = random.choice(child2)

    # Chooses a random position excluding the original position of the selected gene
    new_position1 = random.choice([i for i in range(len(child1)) if child1[i] != gene1])
    new_position2 = random.choice([i for i in range(len(child2)) if child2[i] != gene2])

    # Removes the selected gene from its original position
    old_position1 = child1.index(gene1)
    del child1[old_position1]
    old_position2 = child2.index(gene2)
    del child2[old_position2]

    # Inserts the gene back in at the new randomly assigned position
    child1.insert(new_position1, gene1)
    child2.insert(new_position2, gene2)

    # Ensures mutation adheres to constraint that each city (gene) must be expressed once
    assert len(set(child1)) == len(child1), "City uniqueness violated after mutation"
    assert len(set(child2)) == len(child2), "City uniqueness violated after mutation"

    return [child1, child2]

def InversionMutation(offspring):
    '''
    PARAMETERS: 
    - offspring: the tuple of offspring solutions after crossover to be mutated
    DESCRIPTION: 
    - Conducts inversion mutation on the two offspring 
    RETURNS: 
    - Tuple of routes (genes) of two offspring after mutation
    '''

    # Stores genes of parents in variables
    child1 = offspring[0]
    child2 = offspring[1]

    # Defines random subset of genome for child1
    start1 = random.randint(0, len(child1)-1)
    stop1 = random.randint(start1, len(child1))

    # Defines random subset of genome for child2
    start2 = random.randint(0, len(child2)-1)
    stop2 = random.randint(start2, len(child2))

    # We Invert the order of genes in the randomly generated subset in each child in offspring
    subset1 = list(reversed(child1[start1:stop1]))
    subset2 = list(reversed(child2[start2:stop2]))

    # Handle the segments outside the selected intervals
    child1 = child1[:start1]+subset1+child1[stop1:]
    child2 = child2[:start2]+subset2+child2[stop2:]

    # Ensures mutation adheres to constraint that each city (gene) must be expressed once
    assert len(set(child1)) == len(child1), "City uniqueness violated after mutation"
    assert len(set(child2)) == len(child2), "City uniqueness violated after mutation"

    return [child1, child2]

In [15]:
# ---- REPLACEMENT ----

def PopulationUpdate(D, population, offspring, cities):
    """
    PARAMETERS: 
    - D: Distance Matrix
    - population: list of lists representing candidate solutions
    - offspring: tuple of lists of chosen mutated offspring
    - cities: number of cities
    DESCRIPTION: 
    - Updates the population of solutions by swapping the worst (highest cost) solutions with the mutated offspring
    RETURNS: 
    - Dataframe containing the routes and costs of the entire updated population
    """
    
    # Calculates the cost of each individual solution in the population
    population_costs = PopulationCosts(D, population, cities)

    # Finds the indices of the N worst solutions in the population where N is defined by the amount of offspring we have
    worst_solutions = np.argsort(population_costs)[-len(offspring):]

    # Replaces the worst individuals with the new offspring
    for i, index in enumerate(worst_solutions):
        population[index] = offspring[i]

    # Updates the DataFrame with the modified population
    df = pd.DataFrame({'route': population, 'cost': PopulationCosts(D, population, cities)})

    return df